In [ ]:
import random
import time
from neo4j import GraphDatabase

## Establish Neo4j-Python connection and import the csv 
### Change the file path in the file_path variable to load a different csv

In [ ]:
# Neo4j credentials and connection URI
uri = "neo4j://localhost:7687"  # Replace with your URI
username = "neo4j"  # Replace with your username
password = "12345678"  # Replace with your password

# Initialize the Neo4j driver
driver = GraphDatabase.driver(uri, auth = (username, password))
file_path = 'file:///1000nodes_2.5e-2.csv'   # change csv here 


# Function to import csv using cypher query
def import_csv(tx, file_path):
    try:     
        query = """
        LOAD CSV WITH HEADERS FROM $file_path AS row              
        MERGE (Source:Node {name: row.Source})
        MERGE (Target:Node {name: row.Target})
        MERGE (Source)-[:CONNECTED {weight: toInteger(row.Weight)}]->(Target);
        """

        tx.run(query, file_path=file_path)
        print("CSV data import query executed.")
    except Exception as e:
        print(f"Error during CSV import: {e}")

### Implementation of Dijkstra's algorithm using APOC library in Neo4j

In [ ]:
# Main function for implementation of Dijkstra's algorithm using APOC library in Neo4j

def find_shortest_path(tx, start_node, end_node):
    try:
        start_time = time.time()  # Start timing

        query = """
        MATCH (start:Node {name: $start_node}), (end:Node {name: $end_node})
        CALL apoc.algo.dijkstra(start, end, 'CONNECTED>', 'weight') YIELD path, weight
        RETURN [node in nodes(path) | node.name] AS nodes, weight
        """
        result = tx.run(query, start_node = start_node, end_node = end_node)

        end_time = time.time()  # End timing
        elapsed_time = end_time - start_time  # Calculate elapsed time

        return result.single(), elapsed_time
    except Exception as e:
        print(f"Error during shortest path search: {e}")

        
# Function to get random nodes
def get_random_node(tx, file_path):
    try:
        query = "MATCH (n:Node) RETURN n.name as node_name ORDER BY rand() LIMIT 1"
        result = tx.run(query, file_path = file_path)
        return result.single()["node_name"]
    except Exception as e:
        print(f"Error getting random node: {e}")

### 100 random traversals  

In [ ]:
csv_file_path = file_path  

# Perform 100 random traversals and calculate average time
num_traversals = 100

try:
    with driver.session() as session:
        # Import CSV data into the graph
        session.execute_write(import_csv, csv_file_path)
        
        total_elapsed_time = 0    # Initialize time variable so as to add the time of each traversal to it

        for _ in range(num_traversals):
            start_node = get_random_node(session, csv_file_path)
            print()
            end_node = get_random_node(session, csv_file_path)

            record, elapsed_time = session.execute_read(find_shortest_path, start_node, end_node)   # Main algorithm call

            if record:
                print(f"Shortest path from {start_node} to {end_node}: {record['nodes']}, Weight: {record['weight']}")
                print(f"Time elapsed: {elapsed_time:.4f} seconds")
                total_elapsed_time += elapsed_time
            else:
                print(f"No path found from {start_node} to {end_node}")

    average_time = total_elapsed_time / num_traversals    # Average time of 100 traversals                               
    print(f"Average time for {num_traversals} traversals: {average_time:.4f} seconds")

except Exception as e:
    print(f"An error occurred in the main block: {e}")

finally:
    driver.close()